In [1]:
import cv2
import fitz
import torch
from ultralytics import YOLO
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

print("cv2 version:", cv2.__version__)
print("PyMuPDF version:", fitz.__doc__.splitlines()[0])
print("PyTorch version:", torch.__version__)


cv2 version: 4.12.0
PyMuPDF version: PyMuPDF 1.26.6: Python bindings for the MuPDF 1.26.11 library (rebased implementation).
PyTorch version: 2.9.1+cu128


In [2]:
from pathlib import Path

for split in ["train", "valid", "test"]:
    label_dir = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/QR Code.v2i.yolov11") / split / "labels"
    if not label_dir.exists():
        continue
    for txt in label_dir.glob("*.txt"):
        lines = txt.read_text().strip().splitlines()
        new_lines = []
        for ln in lines:
            if not ln.strip():
                continue
            parts = ln.split()
            parts[0] = "2"      # qr_code index
            new_lines.append(" ".join(parts))
        txt.write_text("\n".join(new_lines))


In [3]:
from pathlib import Path

for split in ["train", "valid", "test"]:
    label_dir = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/QR-scanner.v2140i.yolov11") / split / "labels"
    if not label_dir.exists():
        continue
    for txt in label_dir.glob("*.txt"):
        lines = txt.read_text().strip().splitlines()
        new_lines = []
        for ln in lines:
            if not ln.strip():
                continue
            parts = ln.split()
            parts[0] = "2"      # qr_code index
            new_lines.append(" ".join(parts))
        txt.write_text("\n".join(new_lines))


In [4]:
# # Cell 0: create merged data.yaml for 4 datasets

# from pathlib import Path
# import yaml

# ROOT_DIR = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
# DATA_ROOT = ROOT_DIR / "data"

# # original dataset roots
# DS_IDP          = DATA_ROOT / "IDP_stamp_signature_detection.v2i.yolov8"
# DS_QR_CODE      = DATA_ROOT / "QR Code.v2i.yolov11"
# DS_SIG_STAMPS   = DATA_ROOT / "Signature and Stamps.v3i.yolov8"
# DS_QR_SCANNER   = DATA_ROOT / "QR-scanner.v2140i.yolov11"

# # paths to image folders
# train_dirs = [
#     str(DS_IDP        / "train" / "images"),
#     str(DS_QR_CODE    / "train" / "images"),
#     str(DS_SIG_STAMPS / "train" / "images"),
#     str(DS_QR_SCANNER / "train" / "images"),
# ]

# val_dirs = [
#     str(DS_IDP        / "valid" / "images"),
#     str(DS_QR_CODE    / "valid" / "images"),
#     str(DS_SIG_STAMPS / "valid" / "images"),
#     str(DS_QR_SCANNER / "valid" / "images"),
# ]

# # (optional) test dirs if you want
# test_dirs = [
#     str(DS_IDP        / "test" / "images"),
#     str(DS_QR_CODE    / "test" / "images"),
#     str(DS_SIG_STAMPS / "test" / "images"),
#     str(DS_QR_SCANNER / "test" / "images"),
# ]

# MERGED_YAML = DATA_ROOT / "data_merged_4sets.yaml"

# merged_cfg = {
#     # Ultralytics accepts lists of dirs or globs
#     "train": train_dirs,
#     "val":   val_dirs,
#     "test":  test_dirs,

#     # IMPORTANT: you must have already remapped labels to match this
#     "nc": 3,
#     "names": ["signature", "stamp", "qr_code"],
# }

# with open(MERGED_YAML, "w") as f:
#     yaml.safe_dump(merged_cfg, f, sort_keys=False)

# print("Merged data.yaml written to:", MERGED_YAML)


In [5]:
# Cell 1: train a single YOLO11 model on merged 4 datasets

from ultralytics import YOLO
from pathlib import Path

ROOT_DIR = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT = ROOT_DIR / "runs"
RUNS_ROOT.mkdir(parents=True, exist_ok=True)

MERGED_YAML = ROOT_DIR / "data" / "data_merged_4sets.yaml"

EPOCHS     = 50        # increase if you have time
IMG_SIZE   = 1024
BATCH_SIZE = 8
DEVICES    = [2, 3]    # your two GPUs

print("Training one merged model with:")
print(" data:", MERGED_YAML)
print(" epochs:", EPOCHS)
print(" imgsz:", IMG_SIZE)
print(" batch:", BATCH_SIZE)
print(" devices:", DEVICES)

# fresh YOLO11 model
model = YOLO("yolo11m.pt")   # or yolo11l.pt if you want larger

run_name = "yolo11_merged_4datasets"
results = model.train(
    data=str(MERGED_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICES,
    project=str(RUNS_ROOT),
    name=run_name,
    exist_ok=True,
)

# print("Training finished. Run directory:", results.save_dir)


Training one merged model with:
 data: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets.yaml
 epochs: 50
 imgsz: 1024
 batch: 8
 devices: [2, 3]
Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:2 (NVIDIA GeForce RTX 4090, 24081MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets.yaml, degrees=0.0, deterministic=True, device=2,3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, io

In [6]:
# Cell 2: copy best weights of merged model into models/ folder

import shutil
from pathlib import Path

ROOT_DIR   = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT  = ROOT_DIR / "runs"
MODELS_DIR = ROOT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

run_name   = "yolo11_merged_4datasets"
run_dir    = RUNS_ROOT / run_name
weights_dir = run_dir / "weights"

best_pt = weights_dir / "best.pt"
last_pt = weights_dir / "last.pt"

if best_pt.exists():
    chosen = best_pt
    print("Using best.pt:", chosen)
elif last_pt.exists():
    chosen = last_pt
    print("best.pt not found, using last.pt:", chosen)
else:
    raise FileNotFoundError(f"No weights found in {weights_dir}")

# final name for your merged model
dst = MODELS_DIR / "best_yolo11_merged_4datasets.pt"
shutil.copy(str(chosen), str(dst))
print("Model weights copied to:", dst)


Using best.pt: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/runs/yolo11_merged_4datasets/weights/best.pt
Model weights copied to: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/models/best_yolo11_merged_4datasets.pt


In [7]:
# Cell: Evaluate merged model on TEST split and store metrics

from ultralytics import YOLO
from pathlib import Path

ROOT_DIR    = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
MERGED_YAML = ROOT_DIR / "data" / "data_merged_4sets.yaml"
MODEL_PATH  = ROOT_DIR / "models" / "best_yolo11_merged_4datasets.pt"

model = YOLO(str(MODEL_PATH))

# Evaluate on TEST split
metrics_test = model.val(
    data=str(MERGED_YAML),
    split="test",       # <-- use test split from your YAML
    imgsz=1024,
    device=[2, 3],
    verbose=True,
)

# Grab summary metrics (global, across all classes)
test_precision   = float(metrics_test.box.mp)       # mean precision over IoU=0.5:0.95
test_recall      = float(metrics_test.box.mr)       # mean recall
test_map50_95    = float(metrics_test.box.map)      # mAP@[0.5:0.95]
test_map50       = float(metrics_test.box.map50)    # mAP@0.5

print("TEST metrics:")
print(f"  precision (mp)     = {test_precision:.4f}")
print(f"  recall (mr)        = {test_recall:.4f}")
print(f"  mAP@0.5:0.95 (map) = {test_map50_95:.4f}")
print(f"  mAP@0.5 (map50)    = {test_map50:.4f}")


Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:2 (NVIDIA GeForce RTX 4090, 24081MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24081MiB)
YOLO11m summary (fused): 125 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3719.1±1671.7 MB/s, size: 60.6 KB)
val: Scanning /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/QR Code.v2i.yolov11/test/labels... 383 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 383/383 3.3Kit/s 0.1s<0.1s
val: New cache created: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/QR Code.v2i.yolov11/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 253, len(boxes) = 710. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P       

In [8]:
# # Cell: initialize YOLO11 model

# from ultralytics import YOLO
# import torch

# # choose variant: yolo11n.pt, yolo11s.pt, yolo11m.pt, ...
# BASE_MODEL = "yolo11m.pt"

# model = YOLO(BASE_MODEL)

# print("Using base YOLO model:", BASE_MODEL)
# print("CUDA available:", torch.cuda.is_available())


In [9]:
# # Cell 1: train YOLO11 sequentially on all 4 datasets (no file copying here)

# from pathlib import Path
# from ultralytics import YOLO

# EPOCHS_PER_DATASET = 5      # adjust as you like
# IMG_SIZE          = 1024
# BATCH_SIZE        = 8
# DEVICES           = [2, 3]  # your GPUs

# RUNS_ROOT = ROOT_DIR / "runs"
# RUNS_ROOT.mkdir(parents=True, exist_ok=True)

# print("Training config:")
# print("  epochs per dataset:", EPOCHS_PER_DATASET)
# print("  imgsz:", IMG_SIZE)
# print("  batch:", BATCH_SIZE)
# print("  devices:", DEVICES)
# print("  runs dir:", RUNS_ROOT)

# # Will hold best weight path per dataset and final path
# best_weights_paths = {}
# final_best_path = None

# for ds_name, data_yaml in DATASETS.items():
#     print("=" * 80)
#     print(f"Starting training on dataset: {ds_name}")
#     print(f"  data.yaml: {data_yaml}")

#     # make a clean run name (no spaces)
#     run_name = f"yolo11_{ds_name}".replace(" ", "_")
#     save_dir = RUNS_ROOT / run_name
#     print(f"  run dir: {save_dir}")

#     # Train – NOTE: we do NOT use the return value
#     model.train(
#         data=str(data_yaml),
#         epochs=EPOCHS_PER_DATASET,
#         imgsz=IMG_SIZE,
#         batch=BATCH_SIZE,
#         device=DEVICES,
#         project=str(RUNS_ROOT),
#         name=run_name,
#         exist_ok=True,   # reuse same folder if it exists
#     )

#     # Find best / last weights from the known run directory
#     weights_dir = save_dir / "weights"
#     best_pt = weights_dir / "best.pt"
#     last_pt = weights_dir / "last.pt"

#     if best_pt.exists():
#         best_path = best_pt
#         print(f"[{ds_name}] Using best.pt: {best_path}")
#     elif last_pt.exists():
#         best_path = last_pt
#         print(f"[{ds_name}] best.pt missing, using last.pt: {best_path}")
#     else:
#         raise FileNotFoundError(f"No weights found in {weights_dir}")

#     # store path and prepare for next dataset
#     best_weights_paths[ds_name] = str(best_path)
#     final_best_path = best_path

#     # load best weights before next dataset
#     model = YOLO(str(best_path))
#     print(f"Loaded best weights for next stage: {best_path}")

# print("=" * 80)
# print("Finished multi-dataset training.")
# print("Final best weights path:", final_best_path)


In [10]:
# # Cell 2: copy trained weights into models/ directory

# import shutil
# from pathlib import Path

# MODELS_DIR = ROOT_DIR / "models"
# MODELS_DIR.mkdir(parents=True, exist_ok=True)

# # Safety check
# if "final_best_path" not in globals() or final_best_path is None:
#     raise RuntimeError("Run the training cell first so final_best_path is defined.")

# # 1) Save the final multi-dataset model
# final_dst = MODELS_DIR / "best_yolo11_multi_doc.pt"
# shutil.copy(str(final_best_path), str(final_dst))
# print("Saved final multi-dataset model to:", final_dst)

# # 2) (Optional) also save per-dataset best models with separate names
# for ds_name, path_str in best_weights_paths.items():
#     src = Path(path_str)
#     if not src.exists():
#         print(f"[WARN] Best weights for {ds_name} not found at {src}, skipping.")
#         continue

#     # e.g. best_yolo11_IDP_stamp_signature.pt, etc.
#     safe_name = ds_name.replace(" ", "_")
#     dst = MODELS_DIR / f"best_yolo11_{safe_name}.pt"
#     shutil.copy(str(src), str(dst))
#     print(f"Copied best weights for {ds_name} -> {dst}")


## V1

In [11]:
# Cell 1: train a single YOLO11 model on merged 4 datasets (v1)

from ultralytics import YOLO
from pathlib import Path

ROOT_DIR  = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT = ROOT_DIR / "runs"
RUNS_ROOT.mkdir(parents=True, exist_ok=True)

# NEW: use the v1 yaml
MERGED_YAML_V1 = ROOT_DIR / "data" / "data_merged_4sets_v1.yaml"

EPOCHS     = 50        # same as before
IMG_SIZE   = 1024
BATCH_SIZE = 8
DEVICES    = [2, 3]    # your two GPUs

print("Training one merged model (v1) with:")
print(" data:", MERGED_YAML_V1)
print(" epochs:", EPOCHS)
print(" imgsz:", IMG_SIZE)
print(" batch:", BATCH_SIZE)
print(" devices:", DEVICES)

# fresh YOLO11 model
model = YOLO("yolo11m.pt")   # or yolo11l.pt if you want larger

# NEW: v1 run name
run_name_v1 = "yolo11_merged_4datasets_v1"

results = model.train(
    data=str(MERGED_YAML_V1),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICES,
    project=str(RUNS_ROOT),
    name=run_name_v1,
    exist_ok=True,
)

# Optional: sanity print if results is not None
# if results is not None:
#     print("Training finished. Run directory:", results.save_dir)


Training one merged model (v1) with:
 data: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets_v1.yaml
 epochs: 50
 imgsz: 1024
 batch: 8
 devices: [2, 3]
Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:2 (NVIDIA GeForce RTX 4090, 24081MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets_v1.yaml, degrees=0.0, deterministic=True, device=2,3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int

In [12]:
# Cell 2: copy best weights of merged v1 model into models/ folder

import shutil
from pathlib import Path

ROOT_DIR   = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT  = ROOT_DIR / "runs"
MODELS_DIR = ROOT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# NEW: v1 run name and dir
run_name_v1 = "yolo11_merged_4datasets_v1"
run_dir_v1  = RUNS_ROOT / run_name_v1
weights_dir = run_dir_v1 / "weights"

best_pt = weights_dir / "best.pt"
last_pt = weights_dir / "last.pt"

if best_pt.exists():
    chosen = best_pt
    print("Using best.pt:", chosen)
elif last_pt.exists():
    chosen = last_pt
    print("best.pt not found, using last.pt:", chosen)
else:
    raise FileNotFoundError(f"No weights found in {weights_dir}")

# NEW: final name for your merged v1 model
dst = MODELS_DIR / "best_yolo11_merged_4datasets_v1.pt"
shutil.copy(str(chosen), str(dst))
print("Model weights copied to:", dst)


Using best.pt: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/runs/yolo11_merged_4datasets_v1/weights/best.pt
Model weights copied to: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/models/best_yolo11_merged_4datasets_v1.pt


## V2


In [13]:
# Cell 1: train a single YOLO11 model on merged 4 datasets (v2)

from ultralytics import YOLO
from pathlib import Path

ROOT_DIR  = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT = ROOT_DIR / "runs"
RUNS_ROOT.mkdir(parents=True, exist_ok=True)

# NEW: use the v2 yaml
MERGED_YAML_V2 = ROOT_DIR / "data" / "data_merged_4sets_v2.yaml"

EPOCHS     = 50        # same as before
IMG_SIZE   = 1024
BATCH_SIZE = 8
DEVICES    = [2, 3]    # your two GPUs

print("Training one merged model (v2) with:")
print(" data:", MERGED_YAML_V2)
print(" epochs:", EPOCHS)
print(" imgsz:", IMG_SIZE)
print(" batch:", BATCH_SIZE)
print(" devices:", DEVICES)

# fresh YOLO11 model
model = YOLO("yolo11m.pt")   # or yolo11l.pt if you want larger

# v2 run name
run_name_v2 = "yolo11_merged_4datasets_v2"

results = model.train(
    data=str(MERGED_YAML_V2),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=DEVICES,
    project=str(RUNS_ROOT),
    name=run_name_v2,
    exist_ok=True,
)

# Optional:
# if results is not None:
#     print("Training finished. Run directory:", results.save_dir)


Training one merged model (v2) with:
 data: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets_v2.yaml
 epochs: 50
 imgsz: 1024
 batch: 8
 devices: [2, 3]
Ultralytics 8.3.228 🚀 Python-3.10.19 torch-2.9.1+cu128 CUDA:2 (NVIDIA GeForce RTX 4090, 24081MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24081MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/data/data_merged_4sets_v2.yaml, degrees=0.0, deterministic=True, device=2,3, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int

In [14]:
# Cell 2: copy best weights of merged v2 model into models/ folder

import shutil
from pathlib import Path

ROOT_DIR   = Path("/home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon")
RUNS_ROOT  = ROOT_DIR / "runs"
MODELS_DIR = ROOT_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# v2 run name and dir
run_name_v2 = "yolo11_merged_4datasets_v2"
run_dir_v2  = RUNS_ROOT / run_name_v2
weights_dir = run_dir_v2 / "weights"

best_pt = weights_dir / "best.pt"
last_pt = weights_dir / "last.pt"

if best_pt.exists():
    chosen = best_pt
    print("Using best.pt:", chosen)
elif last_pt.exists():
    chosen = last_pt
    print("best.pt not found, using last.pt:", chosen)
else:
    raise FileNotFoundError(f"No weights found in {weights_dir}")

# final name for your merged v2 model
dst = MODELS_DIR / "best_yolo11_merged_4datasets_v2.pt"
shutil.copy(str(chosen), str(dst))
print("Model weights copied to:", dst)


Using best.pt: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/runs/yolo11_merged_4datasets_v2/weights/best.pt
Model weights copied to: /home/gpuhead-1/Desktop/Makhmud/AISEC_hackathon/models/best_yolo11_merged_4datasets_v2.pt
